In [1]:
using StatsBase

const xopen = 1
const xtree = 2
const xlumb = 3

function readfile(filename)
    lines = readlines(filename)
    hcat([collect(line) for line in lines]...) |> permutedims
end

function visualize(area)
    foreach(row -> println(join(A[row,:])), 1:size(A,1))
    area
end

function incr!(area, cnt, x, y)
    if area[y,x] == '.'
        cnt[xopen] += 1
    elseif area[y,x] == '|'
        cnt[xtree] += 1
    else # lumber
        cnt[xlumb] += 1
    end
end

function countadj!(x, y, w, h, area, cnt)
    x-1 > 0 && y-1 > 0 &&   incr!(area, cnt, x-1, y-1)
    x-1 > 0 &&              incr!(area, cnt, x-1, y)
    x-1 > 0 && y+1 <= h &&  incr!(area, cnt, x-1, y+1)
    y-1 > 0 &&              incr!(area, cnt, x,   y-1)
    y+1 <= h &&             incr!(area, cnt, x,   y+1)
    x+1 <= w && y-1 > 0 &&  incr!(area, cnt, x+1, y-1)
    x+1 <= w &&             incr!(area, cnt, x+1, y) 
    x+1 <= w && y+1 <= h && incr!(area, cnt, x+1, y+1)
end

function generate!(src, dst, w, h)
    cnt = fill(0, 3)
    for x = 1:w, y = 1:h
        cnt[xopen] = cnt[xtree] = cnt[xlumb] = 0
        countadj!(x, y, w, h, src, cnt)
        dst[y, x] = src[y, x]
        if src[y, x] == '.'
            cnt[xtree] >= 3 && (dst[y, x] = '|')
        elseif src[y, x] == '|'
            cnt[xlumb] >= 3 && (dst[y, x] = '#')
        elseif src[y, x] == '#'
            !(cnt[xlumb] >=1 && cnt[xtree] >= 1) && (dst[y, x] = '.')
        end
    end
end

function valueof2(A, n)
    h, w = size(A)
    B = copy(A)
    A = copy(A)
    for i in 1:n
        if isodd(i) 
            generate!(A, B, w, h)
        else
            generate!(B, A, w, h)
        end
    end
    C = isodd(n) ? B : A
    s = C |> countmap
    (value = s['#'] * s['|'], area = C)
end

A = readfile("input18_sample.txt") 
A |> visualize 
v = valueof2(A, 10)
println()
println(v.value)
v.area |> visualize
;

.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.

1147
.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.


In [2]:
A = readfile("input18_sample.txt") 
@time valueof2(A, 10)
@time valueof2(A, 10)
@time valueof2(A, 10)
@time valueof2(A, 10)
@time valueof2(A, 10)

  0.000076 seconds (21 allocations: 2.875 KiB)
  0.000058 seconds (21 allocations: 2.875 KiB)
  0.000055 seconds (21 allocations: 2.875 KiB)
  0.000054 seconds (21 allocations: 2.875 KiB)
  0.000055 seconds (21 allocations: 2.875 KiB)


(value = 1147, area = ['.' '|' … '.' '.'; '|' '|' … '.' '.'; … ; '|' '|' … '|' '|'; '|' '|' … '|' '|'])

In [3]:
A = readfile("input18_sample.txt") 
@time valueof2(A, 10)

A = readfile("input18_sample.txt") 
@time valueof2(A, 10)

A = readfile("input18_sample.txt") 
@time valueof2(A, 10)


  0.000080 seconds (21 allocations: 2.875 KiB)
  0.000136 seconds (21 allocations: 2.875 KiB)
  0.000057 seconds (21 allocations: 2.875 KiB)


(value = 1147, area = ['.' '|' … '.' '.'; '|' '|' … '.' '.'; … ; '|' '|' … '|' '|'; '|' '|' … '|' '|'])

In [4]:
# Part 2.
# Look for pattern
orig = readfile("input18.txt")
for n in 1000:1000:10000
    A = copy(orig)
    v = valueof2(A, n)
    println(n, "==> ", v.value)
end

1000==> 177004
2000==> 219240
3000==> 205120
4000==> 166753
5000==> 194370
6000==> 223401
7000==> 193434
8000==> 177004
9000==> 219240
10000==> 205120


In [5]:
# It looks like a cylce every 70000 iterations
# need to reach 1_000_000_000.  Just guess & bisect.
50_000 + 70_000 * 14285

1000000000